In [87]:
%matplotlib inline
#The line above is necesary to show Matplotlib's plots inside a Jupyter Notebook

import os
import numpy as np
import matplotlib.pyplot as plt
import imageio
import random
import cv2

from PIL import Image, ImageDraw, ImageFont
from pathlib import Path
import string

In [88]:
#生成字母掩码, 黑色是前景，白色是背景
def WriteChar2Img(letter, width, height, font, fontSize):
    img = Image.new('RGB', (width, height), 'white')
    d = ImageDraw.Draw(img)
    letterWidth, letterHeight = d.textsize(letter, font)
    d.text(((width-letterWidth)//2, (height-letterHeight)//3), letter, fill=(0, 0, 0), font=font)
    img.save(f'images/{letter}.png')

def PrepareAlphabet(letters):
    fontSize= (width+height)*40//100
    font = ImageFont.truetype("arial.ttf", fontSize)
    for l in letters:
        if Path(f'images/{l}.png').is_file() == False:
            WriteChar2Img(l, width, height, font, fontSize)

In [89]:
#自己照片
def PhotoToMask(photoName, width, height):
    img = cv2.imread(f'images/{photoName}')
    #注意微信里面只能使用表情，而且小于1M
    img = cv2.resize(img, (width//2, height//2), interpolation = cv2.INTER_AREA)
    standardName = f'images/_{photoName}' 
    cv2.imwrite(standardName, img)
    
    #plt.imshow(img)
    ##plt.show()
    
    return standardName

In [90]:
# 根据字母的形状, 将字母转化为多个随机点
def get_masked_data(letter, intensity=2):
    # 多个随机点填充字母
    random.seed(420)
    x = []
    y = []

    for i in range(intensity):
        x = x + random.sample(range(0, width), width)
        y = y + random.sample(range(0, height), height)

    if letter == ' ':
        return x, y

    # 获取图片的mask
    mask = cv2.imread(f'images/{letter}.png', 0)
    mask = cv2.flip(mask, 0)

    # 检测点是否在mask中
    result_x = []
    result_y = []
    for i in range(len(x)):
        if (mask[y[i]][x[i]]) == 0:
            result_x.append(x[i])
            result_y.append(y[i])

    # 返回x,y
    return result_x, result_y


# 将文字切割成一个个字母
def text_to_data(txt, repeat=True, intensity=2):
    print('将文本转换为数据\n')
    letters = []
    for i in txt:
        letters.append(get_masked_data(i, intensity=intensity))
    # 如果repeat为1时,重复第一个字母
    if repeat:
        letters.append(get_masked_data(txt[0], intensity=intensity))
    return letters

In [91]:
def build_gif(coordinates_lists, gif_name='movie', n_frames=10, bg_color='#95A4AD',
              marker_color='#283F4E', marker_size=25):
    print('生成图表\n')
    filenames = []
    for index in np.arange(0, len(coordinates_lists) - 1):
        # 获取当前图像及下一图像的x与y轴坐标值
        x = coordinates_lists[index][0]
        y = coordinates_lists[index][1]

        x1 = coordinates_lists[index + 1][0]
        y1 = coordinates_lists[index + 1][1]

        # 查看两点差值
        while len(x) < len(x1):
            diff = len(x1) - len(x)
            x = x + x[:diff]
            y = y + y[:diff]

        while len(x1) < len(x):
            diff = len(x) - len(x1)
            x1 = x1 + x1[:diff]
            y1 = y1 + y1[:diff]

        # 计算路径
        x_path = np.array(x1) - np.array(x)
        y_path = np.array(y1) - np.array(y)

        for i in np.arange(0, n_frames + 1):
            # 计算当前位置
            x_temp = (x + (x_path / n_frames) * i)
            y_temp = (y + (y_path / n_frames) * i)

            # 绘制图表
            fig, ax = plt.subplots(figsize=(6, 6), subplot_kw=dict(aspect="equal"))
            ax.set_facecolor(bg_color)
            plt.xticks([])  # 去掉x轴
            plt.yticks([])  # 去掉y轴
            plt.axis('off')  # 去掉坐标轴

            plt.scatter(x_temp, y_temp, c=marker_color, s=marker_size)

            plt.xlim(0, width)
            plt.ylim(0, height)

            # 移除框线
            ax.spines['right'].set_visible(False)
            ax.spines['top'].set_visible(False)

            # 网格线
            ax.set_axisbelow(True)
            ax.yaxis.grid(color='gray', linestyle='dashed', alpha=0.7)
            ax.xaxis.grid(color='gray', linestyle='dashed', alpha=0.7)

            # 保存图片
            filename = f'images/frame_{index}_{i}.png'

            if (i == n_frames):
                for i in range(5):
                    filenames.append(filename)

            filenames.append(filename)

            # 保存
            plt.savefig(filename, dpi=96, facecolor=bg_color)
            plt.close()
    print('保存图表\n')
    
    # 生成GIF
    print('生成GIF\n')
    with imageio.get_writer(f'{gif_name}.gif', mode='I') as writer:
        for filename in filenames:
            image = imageio.imread(filename)
            writer.append_data(image)
        
        image = imageio.imread(myLoverPhoto)
        for i in range(10):
            writer.append_data(image)
    print('保存GIF\n')
    
    print('删除图片\n')
    # 删除图片
    for filename in set(filenames):
        os.remove(filename)
    os.remove(myLoverPhoto)

    print('完成')

In [92]:
#### Main()
width=400   #注意微信里面只能使用表情，而且小于1M
height=400
myLoverName = input('请输入女票的字母缩写:')
PrepareAlphabet(myLoverName)
coordinates_lists = text_to_data(myLoverName, repeat=True, intensity=50)
myLoverPhoto = PhotoToMask('me.png', width, height)

build_gif(coordinates_lists,
          gif_name=myLoverName,
          n_frames=7,
          bg_color='#FFC0CB',
          #marker_color='#FFED33',
          marker_color='#FF0000',
          marker_size=1.0)

请输入女票的字母缩写:SUCHENG
将文本转换为数据

生成图表

保存图表

生成GIF

保存GIF

删除图片

完成
